In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [8]:
H_data=pd.read_csv("/content/heart.csv")


In [9]:
H_data.shape

(303, 14)

In [10]:
H_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [11]:
H_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [12]:
X=H_data.drop(columns="target",axis=1)
Y=H_data["target"]

In [13]:
X=np.asarray(X);
Y=np.asarray(Y);


# Comparing the model using deafult hyper parameters


In [14]:
models=[LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)];


In [16]:
def compare_model_cross_validation():
  for model in models:
    cross_v_score=cross_val_score(model,X,Y,cv=5)
    mean_cross_v_score=sum(cross_v_score)/len(cross_v_score)
    cross_val_percentage=round(mean_cross_v_score*100,2)
    print(model,end=': ')
    print(cross_val_percentage)

In [17]:
compare_model_cross_validation()

LogisticRegression(max_iter=1000): 83.15
SVC(kernel='linear'): 82.83
KNeighborsClassifier(): 64.39
RandomForestClassifier(random_state=0): 83.81


# Tuning the Hyper Parameters and selecting the desired model

In [18]:
models_list=[LogisticRegression(max_iter=10000),SVC(),KNeighborsClassifier(),RandomForestClassifier()]

In [20]:
model_hyperparameters={
    'LR':{'C':[1,5,10,20]},
    'SVC':{'kernel':['linear','poly','rbf','sigmoid'],'C':[1,5,10,20]},
    'KNN':{'n_neighbors':[3,5,10]},
    'RFC':{'n_estimators':[10,20,50,100]}
    }

In [24]:
print(model_hyperparameters.keys())
model_keys=list(model_hyperparameters.keys())
print(model_keys)

dict_keys(['LR', 'SVC', 'KNN', 'RFC'])
['LR', 'SVC', 'KNN', 'RFC']


In [25]:
print(model_hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


In [27]:
def model_selection_tuning_hyperparameters(list_of_models,hyperparameter_dictionary):
  res=[];
  i=0;
  for model in list_of_models:
    key=model_keys[i]
    param=hyperparameter_dictionary[key]
    i=i+1;
    classifier=GridSearchCV(model,param,cv=5)
    classifier.fit(X,Y);
    res.append({
        'model':model,
        'best_score': classifier.best_score_,
        'best_parameter':classifier.best_params_
    })

  result_dataframe=pd.DataFrame(res,columns=['model','best_score','best_parameter'])
  return result_dataframe


In [29]:
model_selection_tuning_hyperparameters(models_list,model_hyperparameters)

,model,best_score,best_parameter
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(),0.825027,{'n_estimators': 100}
